In [1]:
### Before running this code, you will need to:
# (1) Go to https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TransitView/nph-visibletbls?dataset=transits
# (2) Upload the target file (if multiple) or enter the planet name (if single). 
#     TOIs/TIC planets may need to be entered by hand if their coordinates aren't known
# (3) Select Earth Observatory. Coordinates in deg:
#     Challis: N lat =44.49833 E long = -114.33388 (alt = 2165.0 ) (4) Set custom start/end date (note in UT)
#     IoIO? 
#     SAAO: select option
#     Boise State: N lat = 43.603600 N E long = -114.33388
# (4) Currently working with a file that has been pre-filtered for observability, but we'll add that here.

#       SAAO 1-m: above 30 deg altitude during entire window
#       SAAO 74 in (Lasedi): complicated function
#       Challis:
#       IoIO:
# (5) To do: I already removed the duplicates by hand (several different ephemerides, for some planets)
#            I filter for transits that have 1+ hours of baseline above 30 deg alt

In [3]:
import numpy as np
# pip install astropy
# conda install astropy
from astropy.time import Time
from astropy.table import Table
import os
# from datetime import datetime, timedelta

from pytz import timezone


# When are we predicting for?

In [34]:
### CHANGE: when = pick a name for when you are running ; also set the file name
##################
when = "Bruneau_Fall25" # the first part should be "IoIO", "Challis", "SAAO", or "BoiseState"
##############
use_obs = when.replace("_","-").split("-")[0]
print("Predictions are for this observatory:",use_obs)

### CHANGE: ephem_file to reflect file name from exoplanet archive output
ephem_file = "ephem_files/transits_HATP37b_2025.10.06_11.21.55.csv"
print("Using ephemeris file",ephem_file)
      
if use_obs == "Challis":
    dusk = "22" 
    dawn = "16"
elif use_obs == "BoiseState":
    dusk = "22" # Filters ingress, account for baseline (should be dusk +1 hr) 
    dawn = "16" # Filters egress, account for baseline (should be dawn -1 hr)
elif use_obs == "TFT":
    dusk = "22"
    dawn = "16"
elif use_obs == "Bruneau":
    dusk = "21"
    dawn = "16"
else:
    print("STOP! We need a valid observatory")

Predictions are for this observatory: Bruneau
Using ephemeris file ephem_files/transits_HATP37b_2025.10.06_11.21.55.csv


In [4]:
# Making output folder
out_folder = when +"/"
if os.path.isdir(out_folder)==False:
    os.makedirs(out_folder)

# Other info on the targets

In [5]:
# Put info table into ephem_files directory
target_info_file = "ephem_files/decaying_planets_info_table.csv"
target_data = Table.read(target_info_file, format='ascii.csv', comment='#')
print(target_data.colnames)
tar_planets = list(target_data["Planet"])
tar_depth = list(target_data["Depth (%)"])
tar_err_5yr = list(target_data["Err (min/5 yr)"])

['Planet', 'V Mag', 'Dec', 'Period (days)', 'Period (days) err', 'Depth (%)', 'R_P R_E', 'R_p_err_plus', 'R_p_err_minus', 'disc_year', 'Err (min/5 yr)', 'Tau_p', 'Years since discovery']


In [6]:
tar_depth_hash = {}
for ii,pp in enumerate(tar_planets):
    tar_depth_hash[pp] = tar_depth[ii]
print(tar_depth_hash)

{'HAT-P-19 b': 2.15, 'HAT-P-32 b': 2.22, 'HD 189733 b': 2.4, 'KELT-23 A b': 1.86, 'TOI-1516 b': 1.75, 'TOI-2046 b': 1.63, 'TrES-1 b': 1.8, 'TrES-2 b': 1.6, 'WASP-2 b': 1.65, 'KPS-1 b': 1.4, 'HATS-24 b': 1.6, 'Qatar-10 b': 1.6, 'WASP-36 b': 1.9, 'Qatar-1 b': 2.1, 'WASP-114 b': 0.9, 'WASP-163 b': 1.5, 'WASP-64 b': 1.5, 'HATS-35 b': 1.1, 'WASP-164 b': 1.6, 'CoRoT-2 b': 2.6, 'WASP-4 b': 2.2, 'TrES-3 b': 2.7, 'WASP-103 b': 1.2, 'WASP-43 b': 2.5, 'WASP-19 b': 2.0, 'WASP-52 b': 2.5, 'HAT-P-36 b': 1.4, 'WASP-5 b': 1.2, 'HAT-P-23 b': 1.3, 'WASP-104 b': 1.5, 'KELT-16 b': 1.1, 'TIC 468574941': 1.7, 'WASP-12 b': 1.0, 'WASP-145 A b': 1.9, 'WASP-50 b': 2.0, 'WASP-173 A b': 1.2, 'TOI-564 b': 0.9, 'HIP 65 A b': 1.5, 'KELT-14 b': 1.3, 'K2-31 b': 2.0, 'WASP-3 b': 1.0, 'WASP-121 b': 1.4, 'WASP-77 A b': 1.7, 'WASP-76 b': 1.2, 'WASP-18 b': 0.9, 'TIC 427761355': 0.9, 'WASP-33 b': 1.1}


# Read in ephemeris file

In [7]:
ephem_data = Table.read(ephem_file, format='ascii.csv', comment='#')
print(ephem_data.colnames)
planet_name_with_dupes = list(ephem_data["planetname"])

['planetname', 'ra_str', 'ra', 'dec_str', 'dec', 'ploturl_dl', 'reflink', 'algorithm', 'isdefault', 'ismostprecise', 'hasmoreprecise', 'eccenwarning', 'phase', 'period', 'transitduration', 'transitdepthdb', 'transitdepthcalc', 'ttv', 'ingresscalendar', 'ingressjd', 'ingressaltitude', 'midpointcalendar', 'midpointjd', 'propmidpointunc', 'midpointairmass', 'midpointaltitude', 'egresscalendar', 'egressjd', 'egressaltitude', 'targetobsstartcalendar', 'targetobsendcalendar', 'durationbeforeevent', 'durationafterevent', 'fractionobservable', 'magnitude_visible']


In [8]:
# If some planets aren't in this list...
all_planets = list(set(planet_name_with_dupes))
print(tar_depth_hash.keys())
for pp in all_planets:
    if pp not in tar_depth_hash.keys():
        tar_depth_hash[pp] = "NA"

dict_keys(['HAT-P-19 b', 'HAT-P-32 b', 'HD 189733 b', 'KELT-23 A b', 'TOI-1516 b', 'TOI-2046 b', 'TrES-1 b', 'TrES-2 b', 'WASP-2 b', 'KPS-1 b', 'HATS-24 b', 'Qatar-10 b', 'WASP-36 b', 'Qatar-1 b', 'WASP-114 b', 'WASP-163 b', 'WASP-64 b', 'HATS-35 b', 'WASP-164 b', 'CoRoT-2 b', 'WASP-4 b', 'TrES-3 b', 'WASP-103 b', 'WASP-43 b', 'WASP-19 b', 'WASP-52 b', 'HAT-P-36 b', 'WASP-5 b', 'HAT-P-23 b', 'WASP-104 b', 'KELT-16 b', 'TIC 468574941', 'WASP-12 b', 'WASP-145 A b', 'WASP-50 b', 'WASP-173 A b', 'TOI-564 b', 'HIP 65 A b', 'KELT-14 b', 'K2-31 b', 'WASP-3 b', 'WASP-121 b', 'WASP-77 A b', 'WASP-76 b', 'WASP-18 b', 'TIC 427761355', 'WASP-33 b'])


In [35]:
ephem_durations_with_dupes = list(ephem_data['transitduration'])
print(min(ephem_durations_with_dupes),max(ephem_durations_with_dupes))
ephem_alt_in_with_dupes = list(ephem_data['ingressaltitude'])
ephem_alt_eg_with_dupes = list(ephem_data['egressaltitude'])
ephem_alt_mid_with_dupes = list(ephem_data['midpointairmass'])
## Get the calendar times for ingress and egress and padded
ephem_in_cal_with_dupes = list(ephem_data['ingresscalendar'])
ephem_midtimes_with_dupes = list(ephem_data["midpointcalendar"])
ephem_eg_cal_with_dupes = list(ephem_data['egresscalendar'])

2.2149132 2.3304


In [36]:
print(ephem_midtimes_with_dupes[0])

10/17/2025 04:30


In [37]:
def eph_to_utc_time(tt):
    parts = tt.split(" ")
    dmy = parts[0]
    time_ut = parts[1]
    date = dmy.split("/")
    reformatted = date[2]+"-"+date[0]+"-"+date[1]+"T"+time_ut+":00"
    return reformatted

In [38]:
ephem_midtimes_utc_with_dupes = [eph_to_utc_time(aa) for aa in ephem_midtimes_with_dupes]
ephem_in_utc_with_dupes = [eph_to_utc_time(aa) for aa in ephem_in_cal_with_dupes]
ephem_eg_utc_with_dupes = [eph_to_utc_time(aa) for aa in ephem_eg_cal_with_dupes]

In [39]:
ephem_Times_with_dupes = Time(ephem_midtimes_utc_with_dupes, format='isot')
ephem_in_Times_with_dupes = Time(ephem_in_utc_with_dupes, format='isot')
ephem_eg_Times_with_dupes = Time(ephem_eg_utc_with_dupes, format='isot')
ephem_midtimes_jd_with_dupes = ephem_Times_with_dupes.jd
ephem_in_jd_with_dupes = ephem_in_Times_with_dupes.jd
ephem_eg_jd_with_dupes = ephem_eg_Times_with_dupes.jd

In [40]:
print(ephem_midtimes_utc_with_dupes[0],ephem_Times_with_dupes[0].jd)

2025-10-17T04:30:00 2460965.6875


In [41]:
print(ephem_in_jd_with_dupes[0], ephem_eg_jd_with_dupes[0])

2460965.640972222 2460965.7333333334


## Get rid of duplicate transits early so it doesn't mess up the statistics

In [42]:
# Assume that any transit of the same planet that is within 4 hour is a duplicate 
# (shortest possible periods are P=4 hr)
has_alt_ephem={}
for ii in range(len(planet_name_with_dupes)):
    dupe_diff=[]; dupe_indices1=[]
    mt1 = ephem_midtimes_jd_with_dupes[ii]
    for jj in range(len(planet_name_with_dupes)):
        if ii != jj:
            mt2 = ephem_midtimes_jd_with_dupes[jj]
            diff = abs(mt1-mt2)
            if diff < 4/24.: # 4 hour
                #print(round(diff*86400),"sec",planet_name[ii],planet_name[jj])
                if planet_name_with_dupes[ii] == planet_name_with_dupes[jj]:
                    print("duplicate!",round(diff*86400),"sec",
                          planet_name_with_dupes[ii],
                          planet_name_with_dupes[jj])
                    dupe_diff.append(round(diff*86400))
                    dupe_indices1.append([ii,jj])
    dupe_indices = sorted([item for sublist in dupe_indices1 for item in sublist])

    # We want to flag these duplicates since they have uncertain ephemerides
    print(ii, dupe_diff)
    if dupe_diff != []:
        has_alt_ephem[ii] = dupe_indices

duplicate! 2040 sec HAT-P-37 b HAT-P-37 b
0 [2040]
duplicate! 2040 sec HAT-P-37 b HAT-P-37 b
1 [2040]
duplicate! 2040 sec HAT-P-37 b HAT-P-37 b
2 [2040]
duplicate! 2040 sec HAT-P-37 b HAT-P-37 b
3 [2040]
duplicate! 2040 sec HAT-P-37 b HAT-P-37 b
4 [2040]
duplicate! 2040 sec HAT-P-37 b HAT-P-37 b
5 [2040]
6 []


In [43]:
## Now make the lists
using_ind=[]
for ii in range(len(planet_name_with_dupes)):
    if ii in has_alt_ephem.keys():
       # print(ii,has_alt_ephem[ii])
        # arbitrarily pick the FIRST prediction (rather than weighting/picking "best")
        use = has_alt_ephem[ii][0]
        if use not in using_ind:
            using_ind.append(use)
    else:
        using_ind.append(ii)
print(len(planet_name_with_dupes),len(using_ind))
print(using_ind)

7 4
[0, 1, 2, 6]


In [44]:
#### USE these non-duplicated lists
planet_name = [planet_name_with_dupes[ii] for ii in using_ind]
ephem_durations = [ephem_durations_with_dupes[ii] for ii in using_ind]
ephem_alt_in = [ephem_alt_in_with_dupes[ii] for ii in using_ind]
ephem_alt_eg = [ephem_alt_eg_with_dupes[ii] for ii in using_ind]
ephem_alt_mid = [ephem_alt_mid_with_dupes[ii] for ii in using_ind]
ephem_midtimes = [ephem_midtimes_with_dupes[ii] for ii in using_ind]
ephem_midtimes_utc = [ephem_midtimes_utc_with_dupes[ii] for ii in using_ind]
ephem_Times = [ephem_Times_with_dupes[ii] for ii in using_ind]
ephem_midtimes_jd = [ephem_midtimes_jd_with_dupes[ii] for ii in using_ind]
print(len(ephem_durations))

planet_ra = [list(ephem_data['ra_str'])[ii] for ii in using_ind]
planet_dec = [list(ephem_data['dec_str'])[ii] for ii in using_ind]
planet_vmag = [list(ephem_data['magnitude_visible'])[ii] for ii in using_ind]

4


## What times are the actual transits, with and without padding?

In [45]:
### If you want to have shorter baseline, you can decrease this number but not below about 0.5
pad_by = 1. # hour

In [46]:
ingress_jd = []; egress_jd = []; padded_start_jd = []; padded_end_jd = []
for ii,tt in enumerate(ephem_Times):
    dur = ephem_durations[ii]
    ingress_jd.append( tt.jd - dur/2./24. )
    egress_jd.append( tt.jd + dur/2./24. )
    padded_start_jd.append( tt.jd - dur/2./24. - pad_by/24.)
    padded_end_jd.append( tt.jd + dur/2./24. + pad_by/24.)

In [47]:
# Sanity checks
print("Ingress",ingress_jd[0])
print("Egress",egress_jd[0])
print("Padded Ingress",padded_start_jd[0])
print("Padded Egress",padded_end_jd[0])

Ingress 2460965.641355975
Egress 2460965.733644025
Padded Ingress 2460965.5996893086
Padded Egress 2460965.7753106914


## Which transits are available on which nights?

In [57]:
### Check: do you want the first day to be the first day listed in the file
use_first_ephem_day = True # only pick false if trying to match another file
if use_first_ephem_day:
    ff = min(ephem_in_cal_with_dupes)
    dd = ff.split(" ")[0].split("/")
    start_adjust = int(dd[1]) - 1
    first_day = dd[2]+"-"+dd[0]+"-"+ f"{start_adjust}"
    print(first_day)
    ll = max(ephem_eg_cal_with_dupes)
    dd = ll.split(" ")[0].split("/")
    end_adjust = int(dd[1]) + 1
    last_day = dd[2]+"-"+dd[0]+"-"+ f"{end_adjust}"
    print(last_day)
### XX check later: this might run into problems with cutting off the first date

2025-10-16
2025-11-29


In [58]:
length_fiducial_night = (24 - int(dusk) + int(dawn))/24.

In [59]:
first_day_starts = Time(first_day + "T"+dusk+":00:00", format='isot')
last_day_starts = Time(last_day + "T"+dusk+":00:00", format='isot')
print(first_day_starts.jd, first_day_starts)
first_day_ends = first_day_starts + length_fiducial_night
last_day_ends = last_day_starts + length_fiducial_night
print(first_day_ends.jd, first_day_ends)


2460965.375 2025-10-16T21:00:00.000
2460966.1666666665 2025-10-17T16:00:00.000


In [60]:
n_days = last_day_starts.jd-first_day_starts.jd
print(n_days)

44.0


In [61]:
days_start=[]; days_end =[]
for ii in range(int(n_days)+1):
    t = first_day_starts.jd + ii
    days_start.append(t )
    days_end.append( first_day_ends.jd+ii )

In [62]:
days_start_utc = Time(days_start,format="jd").isot
date_start_utc = [days_start_utc[xx].split("T")[0] for xx in range(len(days_start_utc))]

In [63]:
print(first_day_starts.jd, first_day_ends.jd)

2460965.375 2460966.1666666665


In [64]:
# Find which day each transit belongs on
transit_on_day=[]
for ii,tt in enumerate(padded_start_jd):
    found=False
    for dd in range(len(days_start)):
        if (tt > days_start[dd]):
            if tt < days_end[dd]:
                transit_on_day.append(dd)
                found=True
    if found==False:
        transit_on_day.append(None)
        print(planet_name[ii],ephem_midtimes[ii],"Not in the time range")

In [65]:
print(transit_on_day)
print(len(transit_on_day))
print(len(ephem_midtimes))

[0, 14, 28, 42]
4
4


In [66]:
transits_per_day=[]
for dd in range(len(days_start)):
    transits_per_day.append(transit_on_day.count(dd))

daily_transit_hash = {}
high_alt_transits = np.full(len(ephem_alt_in), False)
for ii,tt in enumerate(transits_per_day):
    all_daily_transits = []
    for jj,day in enumerate(transit_on_day):
        if day != None:
            if ii==day:
                # determine if "high altitude" target for Unistellars.. (alt >=70)
                if ephem_alt_in[jj] >= 70:
                    high_alt_transits[jj] = True
                elif ephem_alt_eg[jj] >=70:
                    high_alt_transits[jj] = True
                else:
                    pass
                all_daily_transits.append([planet_name[jj],padded_start_jd[jj],ephem_midtimes_jd[jj],padded_end_jd[jj], high_alt_transits[jj]])
    daily_transit_hash[ii] = all_daily_transits

In [67]:
transits_per_day=[]
for dd in range(len(days_start)):
    transits_per_day.append(transit_on_day.count(dd))

In [68]:
### Look here
for ii in range(10):
    print("Days with ",ii," transit(s)", transits_per_day.count(ii))


Days with  0  transit(s) 41
Days with  1  transit(s) 4
Days with  2  transit(s) 0
Days with  3  transit(s) 0
Days with  4  transit(s) 0
Days with  5  transit(s) 0
Days with  6  transit(s) 0
Days with  7  transit(s) 0
Days with  8  transit(s) 0
Days with  9  transit(s) 0


### Of the days with 2+ transits, how many have more than one non-overlapping transits?

In [69]:
print(len(transit_on_day))

4


In [70]:
def do_transits_overlap(start1,end1,start2,end2):
    if (start1>= end2) | (start2>=end1):
        return 0
    else:
        if start1 < start2: # happens first
            return end1 - start2
        elif start2 < start1:
            return end2 - start1
# test
print(do_transits_overlap(0.1,0.2, 0.3, 0.4)) # no
print(do_transits_overlap(0.1,0.35, 0.3, 0.4)) # yes
print(do_transits_overlap(0.4,0.5, 0.3, 0.46)) # no

0
0.04999999999999999
0.06


### Information on each transit happening by day number

In [71]:
# daily_transit_hash = {}
# for ii,tt in enumerate(transits_per_day):
#     all_daily_transits = []
#     for jj,day in enumerate(transit_on_day):
#         if day != None:
#             if ii==day:
#                 all_daily_transits.append([planet_name[jj],padded_start_jd[jj],padded_end_jd[jj]])
#     daily_transit_hash[ii] = all_daily_transits

In [72]:
daily_transit_hash[11]
print(transits_per_day)
print(len(transits_per_day))
print(len(date_start_utc))
print(len(daily_transit_hash.keys()))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
45
45
45


### Get rid of overlap

We can't observe two transits at the same time, so we need to get rid of the ones that are completely overlapping. This will also weed out duplicates from the ephemeris program, which gives several predictions for the same transit for some systems.

In [73]:
#How much overlap is okay?
okay_overlap = 0./60 /24. # 0 minutes
30./60/24 # 30 minutes

0.020833333333333332

In [74]:
no_overlap=[]; 
for ii,tt in enumerate(transits_per_day):
    if tt >1:
        all_daily_transits = []
        for jj,day in enumerate(transit_on_day):
            if day != None:
                if day==ii:
                    #print(planet_name[jj],"start",padded_start_jd[jj],"end",padded_end_jd[jj])
                    all_daily_transits.append([planet_name[jj],padded_start_jd[jj],ephem_midtimes_jd[jj],padded_end_jd[jj]])
        # Now check if transits are overlapping
        for xx in range(len(all_daily_transits)):
            for yy in range(len(all_daily_transits)):
                if xx != yy:
                    if all_daily_transits[xx][0] == all_daily_transits[yy][0]:
                        print("Duplicate! Highly unlikely two transits of same planet on same night", all_daily_transits[xx][0])
                    else:
                         # aim for zero overlap unless pressed (set above)
                        overlap = do_transits_overlap(all_daily_transits[xx][1],all_daily_transits[xx][2],
                                             all_daily_transits[yy][1],all_daily_transits[yy][2])
                        if overlap == None:
                            overlap = 0.
                        if overlap <= okay_overlap:
                            info = [ii, days_start[ii], date_start_utc[ii], sorted([all_daily_transits[xx][0], all_daily_transits[yy][0]])]
                            if info not in no_overlap:
                                no_overlap.append(info)
#                            print(ii, days_start[ii], all_daily_transits[xx][0], all_daily_transits[yy][0],"no overlap")

In [75]:
no_overlap

[]

In [76]:
no_overlap_day_nums = [no_overlap[xx][0] for xx in range(len(no_overlap))]
no_overlap_dates = [no_overlap[xx][2] for xx in range(len(no_overlap))]

In [77]:
groups_of_two=[] # number of consecutive days
groups_of_three=[] # number of consecutive days

In [78]:
for nn in no_overlap_day_nums:
    if nn + 1 in no_overlap_day_nums:
        if nn+2 in no_overlap_day_nums:
            groups_of_three.append([nn,nn+1,nn+2])
for nn in no_overlap_day_nums:
    if nn + 1 in no_overlap_day_nums:
        groups_of_two.append([nn,nn+1])

In [79]:
print(groups_of_three)
print(groups_of_two)

[]
[]


In [80]:
print("Two nights in a row, starting")
for gg in groups_of_two:
    for nn in no_overlap:
        if nn[0] == gg[0]:
            print(nn[2])
print("Three nights in a row, starting")
for gg in groups_of_three:
    for nn in no_overlap:
        if nn[0] == gg[0]:
            print(nn[2])

Two nights in a row, starting
Three nights in a row, starting


### Which planets are on those chunks

In [81]:
objects_in_clumps = []
for gg in groups_of_two:
    for dd in gg:
        for nn in no_overlap:
            if nn[0] == dd:
                for obj in nn[3]:
                    objects_in_clumps.append(obj)
print(set(objects_in_clumps))

set()


In [82]:
all_available_planets = list(set(planet_name))
all_in_clumps = list(set(objects_in_clumps))

In [83]:
print(len(all_available_planets))
print(len(all_in_clumps))
only_single_transits_possible=[]
for pp in all_available_planets:
    if pp not in all_in_clumps:
        only_single_transits_possible.append(pp)

1
0


#### Find the missing singletons

In [84]:
for pp in only_single_transits_possible:
    print(pp)
    for ii in range(len(planet_name)):
        if pp == planet_name[ii]:
            print(ii,padded_start_jd[ii],padded_end_jd[ii])

HAT-P-37 b
0 2460965.5996893086 2460965.7753106914
1 2460979.5871893084 2460979.7628106913
2 2460993.5739948642 2460993.749616247
3 2461007.535477778 2461007.715911111


### Output the full daily transit hash

In [85]:
daily_transit_hash[2]

[]

In [86]:
outfile = out_folder+"Dates_for_all_transits.txt"
print(outfile)
file=open(outfile, "w")
for ii,nn in enumerate(date_start_utc):
    print("Day",ii,date_start_utc[ii],file=open(outfile, "a"))
    for hh in daily_transit_hash[ii]:
        print(hh,file=open(outfile, "a"))


Bruneau_Fall25/Dates_for_all_transits.txt


### Output transit clump days for proposal

In [87]:
all_days_in_clumps = sorted(list(set([item for sublist in groups_of_two for item in sublist])))
all_days_in_clumps

[]

In [88]:
for gg in all_days_in_clumps:
    print(gg)
    print(daily_transit_hash[gg])

In [54]:
## List all no-overlap two transit nights

In [89]:
print(len(no_overlap_day_nums))

0


In [90]:
just_did = []
outfile = out_folder+"Dates_with_two_plus_transits.txt"
file=open(outfile, "w")
for ii,nn in enumerate(no_overlap_day_nums):
    if nn not in just_did:
        print("Day",nn,no_overlap_dates[ii],file=open(outfile, "a"))
        for hh in daily_transit_hash[nn]:
            print(hh,file=open(outfile, "a"))
    # stop duplicates for nights with 3+ transits
    just_did.append(nn)
print(outfile)

Bruneau_Fall25/Dates_with_two_plus_transits.txt


## Output fuller target list

In [91]:
ra_hash = {}; dec_hash={}; vmag_hash={}
for ii,pp in enumerate(planet_name):
    if pp not in ra_hash.keys():
        pos = planet_name.index(pp)
        ra_hash[pp] = planet_ra[pos]
        dec_hash[pp] = planet_dec[pos]
        vmag_hash[pp] = planet_vmag[pos]

In [92]:
in_cal_Time = Time(padded_start_jd, format='jd')
in_cal = in_cal_Time.utc
in_cal.format = 'isot'
mid_cal_Time = Time(ephem_midtimes_jd, format='jd')
mid_cal = mid_cal_Time.utc
mid_cal.format = 'isot'
eg_cal_Time = Time(padded_end_jd, format='jd')
eg_cal = eg_cal_Time.utc
eg_cal.format = 'isot'

In [93]:
outfile = out_folder+"Full_transits_"+when+".txt"
file=open(outfile, "w")
print("\t".join(["Planet", "RA", "Dec", "V", "Depth", "Padded_Start_UT_JD", "Midtime_UT_JD", "Padded_End_UT_JD", 
       "Start_date", "Padded_Start_UT_mtn", "Mid_date", "Mid_time","End_Date", "Padded_End_UT_mtn", "High Altitude Target?"]), file=open(outfile, "a"))
mountain = timezone('US/Mountain')
print(mountain)
fmt = '%H:%M:%S'
for ii,nn in enumerate(date_start_utc):
    print("Day",ii,date_start_utc[ii],":",file=open(outfile, "a"))
    for hh in daily_transit_hash[ii]:
        pp = hh[0]
        in_cal_Time = Time(hh[1], format='jd')
        in_cal = in_cal_Time.utc
        in_cal.format = 'isot'
        in_mtn = in_cal.to_datetime(timezone=mountain)
        mid_cal_Time = Time(hh[2], format='jd')
        mid_cal = mid_cal_Time.utc
        mid_cal.format = 'isot'
        mid_mtn = mid_cal.to_datetime(timezone=mountain)
        eg_cal_Time = Time(hh[3], format='jd')
        eg_cal = eg_cal_Time.utc
        eg_cal.format = 'isot'
        eg_mtn = eg_cal.to_datetime(timezone=mountain)
        print("\t".join([pp, ra_hash[pp], dec_hash[pp], str(vmag_hash[pp]), str(tar_depth_hash[pp]),
              str(hh[1]), str(hh[2]), str(hh[3]), str(in_mtn),str(mid_mtn), str(eg_mtn), str(hh[4])]),
              file=open(outfile, "a"))
print(outfile)

US/Mountain
Bruneau_Fall25/Full_transits_Bruneau_Fall25.txt


In [94]:
pp, ra_hash[pp], dec_hash[pp], str(vmag_hash[pp]), str(tar_depth_hash[pp]), str(hh[1]), str(hh[2]), in_cal, eg_cal

('HAT-P-37 b',
 '18h57m11.05s',
 '51d16m08.84s',
 '13.427',
 'NA',
 '2461007.535477778',
 '2461007.6256944444',
 <Time object: scale='utc' format='isot' value=2025-11-28T00:51:05.280>,
 <Time object: scale='utc' format='isot' value=2025-11-28T05:10:54.720>)

# Output Easy Schedule

In [95]:
outfile = out_folder+"Easy_Schedule_"+when+".txt"
file=open(outfile, "w")
print("\t".join(["Planet", "RA", "Dec", "Start_date", "Padded_Start_UT_mtn", "Mid_date","Mid_time", "End_date", "Padded_End_UT_mtn", "High Altitude Target?"]), file=open(outfile, "a"))
print(mountain)
fmt = '%H:%M:%S'
for ii,nn in enumerate(date_start_utc):
    print("Day",ii,date_start_utc[ii],":",file=open(outfile, "a"))
    for hh in daily_transit_hash[ii]:
        pp = hh[0]
        in_cal_Time = Time(hh[1], format='jd')
        in_cal = in_cal_Time.utc
        in_cal.format = 'isot'
        in_mtn = in_cal.to_datetime(timezone=mountain)
        mid_cal_Time = Time(hh[2], format='jd')
        mid_cal = mid_cal_Time.utc
        mid_cal.format = 'isot'
        mid_mtn = mid_cal.to_datetime(timezone=mountain)
        eg_cal_Time = Time(hh[3], format='jd')
        eg_cal = eg_cal_Time.utc
        eg_cal.format = 'isot'
        eg_mtn = eg_cal.to_datetime(timezone=mountain)
        print("\t".join([pp, ra_hash[pp], dec_hash[pp], str(in_mtn), str(mid_mtn), str(eg_mtn), str(hh[4])]),
              file=open(outfile, "a"))

print(outfile)

US/Mountain
Bruneau_Fall25/Easy_Schedule_Bruneau_Fall25.txt


Output Schedule file for Lauren (Unistellar) all dates in UTC with midtimes

In [96]:
outfile = out_folder+"Unistellar_Schedule_"+when+".txt"
file=open(outfile, "w")
print("\t".join(["Planet", "RA", "Dec", "Start_date", "Padded_Start_UTC", "Mid_date","Mid_time_UTC", "End_date", "Padded_End_UTC", "High Altitude Target?"]), file=open(outfile, "a"))
print(mountain)
fmt = '%H:%M:%S'
for ii,nn in enumerate(date_start_utc):
    # print("Day",ii,date_start_utc[ii],":",file=open(outfile, "a"))
    for hh in daily_transit_hash[ii]:
        pp = hh[0]
        in_cal_Time = Time(hh[1], format='jd')
        in_cal = in_cal_Time.utc
        in_cal.format = 'isot'
        # in_mtn = in_cal.to_datetime(timezone=mountain)
        mid_cal_Time = Time(hh[2], format='jd')
        mid_cal = mid_cal_Time.utc
        mid_cal.format = 'isot'
        # mid_mtn = mid_cal.to_datetime(timezone=mountain)
        eg_cal_Time = Time(hh[3], format='jd')
        eg_cal = eg_cal_Time.utc
        eg_cal.format = 'isot'
        # eg_mtn = eg_cal.to_datetime(timezone=mountain)
        print("\t".join([pp, ra_hash[pp], dec_hash[pp], str(in_cal), str(mid_cal), str(eg_cal), str(hh[4])]),
              file=open(outfile, "a"))

print(outfile)

US/Mountain
Bruneau_Fall25/Unistellar_Schedule_Bruneau_Fall25.txt
